In [6]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from pathlib import Path
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m18/lesson_2/datasets/HR-Employee-Attrition.csv')
attrition_df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [7]:
# Generate our categorical variable lists
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [8]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat, axis=1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/Users/bradleywise/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 499 (1.95 KB)

 Trainable params: 499 (1.95 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.7718 - loss: 0.5685 
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - accuracy: 0.8087 - loss: 0.5138
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - accuracy: 0.8289 - loss: 0.4764
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8317 - loss: 0.4430
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.8333 - loss: 0.4157
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.8411 - loss: 0.3769
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.8322 - loss: 0.3781
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - accuracy: 0.8338 - loss: 0.3543
Epoch 9/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - accuracy: 0.8156 - loss: 0.3798
Epoch 10/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - accuracy: 0.8437 - loss: 0.3491
Epoch 11/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - accuracy: 0.8452 - loss: 0.3362
Epoch 12/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - accuracy: 0.9104 - loss: 0.2161
Epoch 69/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - accuracy: 0.9314 - loss: 0.1958
Epoch 70/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.9311 - loss: 0.1755
Epoch 71/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - accuracy: 0.9276 - loss: 0.1873
Epoch 72/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - accuracy: 0.9348 - loss: 0.1636
Epoch 73/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step - accuracy: 0.9324 - loss: 0.1826
Epoch 74/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.9258 - loss: 0.1786
Epoch 75/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - accuracy: 0.9209 - loss: 0.1832
Epoch 76/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.9279 - loss: 0.1736
Epoch 77/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.9344 - loss: 0.1742
Epoch 78/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - accuracy: 0.9452 - loss: 0.1677
Epoch 79/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 50

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - 10ms/step - accuracy: 0.8832 - loss: 0.4252
Loss: 0.4251995384693146, Accuracy: 0.883152186870575


## Saving the Trained Model

In [16]:
# Set the model's file path
file_path = Path("saved_models/attrition.keras")

# Export your model to a keras file
nn.save(file_path)

## Loading a Trained Model

In [17]:
# Import the required libraries
import tensorflow as tf

In [18]:
# Set the model's file path
file_path = Path("saved_models/attrition.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

TypeError: <class 'keras.src.models.sequential.Sequential'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 55], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'dense_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Dense', 'config': {'name': 'dense', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': [None, 55], 'units': 8, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'conf

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled, y_test, verbose=2)

# Display evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")